In [1]:
import math
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import anndata
import scanpy as sc
import scipy.sparse as sp
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse.csc import csc_matrix
from scipy.sparse.csr import csr_matrix
import pandas as pd
import scipy.stats as st
from torchsummary import summary

/home/sqjiao/.conda/envs/DDPM/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_923019/1927111561.py:15: DeprecationWarning: Please use `csc_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csc` namespace is deprecated.
  from scipy.sparse.csc import csc_matrix
/tmp/ipykernel_923019/1927111561.py:16: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  from scipy.sparse.csr import csr_matrix


In [2]:
from preprocess import construct_interaction,construct_interaction_KNN,preprocess,get_feature,preprocess_adj_sparse,preprocess_adj,mask_nodes_edges
from gae.model import GCNModelVAE_FC
from gae.lossfuc import loss_kl,loss_zinb,loss_CE,loss_nb

In [3]:
# Settings
device = torch.device("cuda:2") if torch.cuda.is_available() else torch.device("cpu")#this should be set to the GPU device you would like to use on your machine
use_cuda=True #set to true if GPU is used 
# seed=3 #random seed
# testNodes=0.1 #fraction of total cells used for testing
# valNodes=0.05 #fraction of total cells used for validation
useSavedMaskedEdges=True #some edges of the adjacency matrices are held-out for validation; set to True to save and use saved version of the edge masks
####1600########
epochs=2000 #number of training epochs
####1600########
saveFreq=200 #the model parameters will be saved during training at a frequency defined by this parameter
lr=0.001 #initial learning rate
weight_decay=0 #regularization term

hidden1=512 #Number of units in hidden layer 1
hidden2=128 #Number of units in hidden layer 2
fc_dim1=512 #Number of units in the fully connected layer of the decoder

dropout=0.01 #neural network dropout term
#human_ovarian_cancer_target
XreconWeight=10  #reconstruction weight of the gene expression
#osmFISH
# XreconWeight=1  #reconstruction weight of the gene expression
#15

# XreconWeight=20
advWeight=2 # weight of the adversarial loss, if used
ridgeL=0.01 #regularization weight of the gene dropout parameter
# ridgeL=5 #regularization weight of the gene dropout parameter
#5
training_sample_X='logminmax' #specify the normalization method for the gene expression input. 'logminmax' is the default that log transforms and min-max scales the expression. 'corrected' uses the z-score normalized and ComBat corrected data from Hu et al. 'scaled' uses the same normalization as 'corrected'.
switchFreq=10 #the number of epochs spent on training the model using one sample, before switching to the next sample
name='get_gae_feature' #name of the model

#provide the paths to save the training log, trained models, and plots, and the path to the directory where the data is stored
#human_ovarian_cancer_target
# logsavepath='./logs/train_gae_human_ovarian_cancer/'+name
# modelsavepath='./models/train_gae_human_ovarian_cancer/'+name
# plotsavepath='./plots/train_gae_human_ovarian_cancer/'+name
#osmFISH
# logsavepath='./logs/train_gae_human_ovarian_cancer/'+name
# modelsavepath='./models/train_gae_human_ovarian_cancer/'+name
# plotsavepath='./plots/train_gae_human_ovarian_cancer/'+name

# logsavepath='./logs/mouse_N06/'+name
# modelsavepath='./models/mouse_N06/'+name
# plotsavepath='./plots/mouse_N06/'+name

datatype='10x'

In [5]:
#human_ovarian_cancer_target
# adata = sc.read_visium('../data/human_ovarian_cancer_target',
#            count_file='Targeted_Visium_Human_OvarianCancer_Pan_Cancer_filtered_feature_bc_matrix.h5'
#            ,source_image_path='Targeted_Visium_Human_OvarianCancer_Pan_Cancer_image.tif'
#                       )
# #osmFISH
# # adata = sc.read_h5ad('../Spatial/Starmap/STARmap_Wang2018Three_1k_mouse_brain_STARmap_data.h5ad')
# adata.var_names_make_unique()

In [4]:
#实验1
adata = sc.read_visium('../data/human_breast_cancer',
           count_file='V1_Breast_Cancer_Block_A_Section_1_filtered_feature_bc_matrix.h5'
           ,source_image_path='V1_Breast_Cancer_Block_A_Section_1_image.tif'
                      )
adata.var_names_make_unique()




/home/sqjiao/.conda/envs/DDPM/lib/python3.10/site-packages/anndata/_core/anndata.py:1900: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/sqjiao/.conda/envs/DDPM/lib/python3.10/site-packages/anndata/_core/anndata.py:1900: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [5]:
print(adata.X.shape)
np.random.seed(42)

matrix  = adata.X.toarray()
nonzero_indices = np.transpose(np.nonzero(matrix))
num_elements_to_zero = int(len(nonzero_indices) * 0.5)
selected_indices = nonzero_indices[np.random.choice(len(nonzero_indices), num_elements_to_zero, replace=False)]
for index in selected_indices:
        matrix[tuple(index)] = 0
adata.X = matrix
print(selected_indices,selected_indices.shape)

(3798, 36601)
[[  668 25440]
 [ 2618 13732]
 [   74 34661]
 ...
 [ 3717 24205]
 [ 2882 15882]
 [  487 22100]] (10676164, 2)


In [8]:
# adata = sc.read_visium('../data/human_colorectal_cancer/',
#            count_file='Targeted_Visium_Human_ColorectalCancer_GeneSignature_filtered_feature_bc_matrix.h5'
#            ,source_image_path='Targeted_Visium_Human_ColorectalCancer_GeneSignature_image.tif'
#                       )
# adata.var_names_make_unique()

# mouse

In [9]:
# data_path = '../data/mouse'
# count_file = 'N06_D2.h5ad'
# source_image_path = 'N06_D2__HE.png'
# adata = sc.read_h5ad(os.path.join(data_path,count_file))
# adata.obsm['spatial']=adata.obs[['img_coord_X','img_coord_Y']].values.astype(np.int)

## 构造图的领接矩阵时，k是影响因素，需要调

In [6]:
#preprocess data
if 'highly_variable' not in adata.var.keys():
    preprocess(adata)

if 'adj' not in adata.obsm.keys():
    if datatype in ['Stereo', 'Slide']:
        construct_interaction_KNN(adata)
    else:    
      # construct_interaction(adata)
        construct_interaction_KNN(adata)

if 'feat' not in adata.obsm.keys():
    get_feature(adata)


/home/sqjiao/.conda/envs/DDPM/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:61: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


Graph constructed!


In [7]:
# matrix  = adata.obsm['feat'].toarray()
# total_elements = matrix.size
# num_elements_to_zero = int(total_elements * 0.60)

# # 随机选择需要置为0的元素的索引
# indices = np.unravel_index(np.random.choice(total_elements, num_elements_to_zero, replace=False), matrix.shape)

# # 将选择的元素置为0
# matrix[indices] = 0
# adata.obsm['feat'] = matrix


In [7]:
#graph feature
features = torch.tensor(adata.obsm['feat'].copy()).to(device)
adj = adata.obsm['adj']
graph_neigh = torch.tensor(adata.obsm['graph_neigh'].copy() + np.eye(adj.shape[0])).to(device)

dim_input = features.shape[1]
dim_output = hidden2

#get adj
if datatype in ['Stereo', 'Slide']:
    #using sparse
    print('Building sparse matrix ...')
    adj_norm = preprocess_adj_sparse(adj).to(device)
    pos_weight = torch.tensor(float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum())
    norm = torch.tensor(adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2))
    adj = torch.tensor(adj+ sp.eye(adj.shape[0])).to(device)
else: 
    # standard version
    # adj_norm = preprocess_adj(adj)
    # adj_norm = torch.FloatTensor(adj_norm).to(device)
    adj_norm = preprocess_adj_sparse(adj).to(device)
    pos_weight = torch.tensor(float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum())
    norm = torch.tensor(adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2))
    adj = torch.tensor(adj+ sp.eye(adj.shape[0])).to(device)


In [12]:
# if not os.path.exists(logsavepath):
#     os.mkdir(logsavepath)
# if not os.path.exists(modelsavepath):
#     os.mkdir(modelsavepath)
# if not os.path.exists(plotsavepath):
#     os.mkdir(plotsavepath)

In [8]:
num_features = features.shape[1]
model = GCNModelVAE_FC(num_features, hidden1,hidden2,fc_dim1, dropout).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

## 是否要加lossA

In [9]:
def train(epochs):
    train_loss_ep=[None]*epochs
    train_loss_kl_ep=[None]*epochs
    train_loss_x_ep=[None]*epochs
    train_loss_a_ep=[None]*epochs
    for epoch in range(epochs):
        # maskedgeres= mask_nodes_edges(features.shape[0],testNodeSize=testNodes,valNodeSize=valNodes,seed=seed)
        # train_nodes_idx,val_nodes_idx,test_nodes_idx = maskedgeres
        
        t = time.time()
        model.train()
        optimizer.zero_grad()
        
        

        adj_recon,mu,logvar,z,features_recon = model(features, adj_norm)
        loss_kl_train=loss_kl(mu, logvar)
        loss_x_train=loss_zinb(features_recon, features,XreconWeight,ridgeL)
        loss_function = nn.MSELoss()
        loss_r_train=loss_function(features_recon[3], features)
        # loss_x_train = loss_nb(features_recon, features,XreconWeight)
        loss_a_train=loss_CE(adj_recon, adj, pos_weight, norm)

        loss=loss_kl_train+loss_x_train+loss_r_train
        # loss=loss_x_train+loss_r_train
        # loss = loss_kl_train+loss_x_train+loss_a_train
        loss.backward()
        optimizer.step()
        train_loss_ep[epoch],train_loss_kl_ep[epoch],train_loss_x_ep[epoch],train_loss_a_ep[epoch]=loss.item(),loss_kl_train.item(),loss_x_train.item(),loss_a_train.item()
        if epoch%saveFreq == 0:
            print(' Epoch: {:04d}'.format(epoch),
                  'loss_train: {:.4f}'.format(loss.item()),
                  'loss_kl_train: {:.4f}'.format(loss_kl_train.item()),
                  'loss_x_train: {:.4f}'.format(loss_x_train.item()),
                  'loss_a_train: {:.4f}'.format(loss_a_train.item()),
                  'time: {:.4f}s'.format(time.time() - t))

            sam = adata.obsm['feat']
            com = features_recon[3].detach().cpu().numpy()
            sam = anndata.AnnData(sam,var = adata[:,adata.var['highly_variable']].var)
            com = anndata.AnnData(com)
            def cal_Percor(original,res):
                Pearson_CoPearson_Cor = pd.Series(index=original.var_names)
                for i in range(res.X.shape[1]):
                    Pearson_CoPearson_Cor[i]=st.pearsonr(original.X[i],res.X[i])[0]
                Pearson_Cor_mean = np.mean(Pearson_CoPearson_Cor)
                return Pearson_CoPearson_Cor,Pearson_Cor_mean
            our_Percor,our_Percor_mean = cal_Percor(sam,com)

            print(our_Percor_mean)
#         if our_Percor_mean>0.72:
#             break
#         if epoch%saveFreq == 0:
#             torch.save(model.cpu().state_dict(), os.path.join(modelsavepath,str(epoch)+'.pt'))
    # with torch.no_grad():
    # torch.save(model.cpu().state_dict(), os.path.join(modelsavepath,'gae.pt'))
    model.to(device).eval()

    adj_recon,mu,logvar,z, features_recon = model(features, adj_norm)

    return train_loss_ep,train_loss_kl_ep,train_loss_x_ep,train_loss_a_ep,z, features_recon

In [10]:
t_ep=time.time()
train_loss_ep,train_loss_kl_ep,train_loss_x_ep,train_loss_a_ep,z, features_recon=train(epochs)
print(' total time: {:.4f}s'.format(time.time() - t_ep))

 Epoch: 0000 loss_train: 5.2840 loss_kl_train: 0.0002 loss_x_train: 4.3238 loss_a_train: 4.9817 time: 1.2625s


/tmp/ipykernel_923019/1318954763.py:45: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  Pearson_CoPearson_Cor[i]=st.pearsonr(original.X[i],res.X[i])[0]


-0.014381331139667563
 Epoch: 0200 loss_train: 1.1283 loss_kl_train: 0.0064 loss_x_train: 1.1136 loss_a_train: 20.5918 time: 0.0495s


/tmp/ipykernel_923019/1318954763.py:45: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  Pearson_CoPearson_Cor[i]=st.pearsonr(original.X[i],res.X[i])[0]


0.5468240154220678
 Epoch: 0400 loss_train: 1.1170 loss_kl_train: 0.0053 loss_x_train: 1.1035 loss_a_train: 16.1485 time: 0.0513s


...

0.5610364460041238
 Epoch: 1600 loss_train: 1.0981 loss_kl_train: 0.0044 loss_x_train: 1.0858 loss_a_train: 8.3574 time: 0.0500s


/tmp/ipykernel_923019/1318954763.py:45: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  Pearson_CoPearson_Cor[i]=st.pearsonr(original.X[i],res.X[i])[0]


0.5632231486835553
 Epoch: 1800 loss_train: 1.0960 loss_kl_train: 0.0045 loss_x_train: 1.0835 loss_a_train: 8.0347 time: 0.0504s


/tmp/ipykernel_923019/1318954763.py:45: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  Pearson_CoPearson_Cor[i]=st.pearsonr(original.X[i],res.X[i])[0]


0.5650276665679116
 total time: 137.8011s


In [16]:
# torch.save(train_loss_ep,os.path.join(logsavepath,'train_loss'))
# torch.save(train_loss_kl_ep,os.path.join(logsavepath,'train_loss_kl'))
# torch.save(train_loss_x_ep,os.path.join(logsavepath,'train_loss_x'))
# torch.save(train_loss_a_ep,os.path.join(logsavepath,'train_loss_a'))

In [11]:
name = 'get_gae_feature'
modelsavepath='./models/human_breast_cancer/'+name
torch.save(model.cpu().state_dict(), os.path.join(modelsavepath,'breast_gae_drop_new050.ckpt'))

In [ ]:
# plt.plot(np.arange(epochs),train_loss_x_ep)
# plt.plot(np.arange(epochs),train_loss_a_ep)
# plt.plot(np.arange(epochs),train_loss_kl_ep)
# plt.legend(['training x recon loss','training a recon loss','training kl loss'],loc='upper right')
# # plt.savefig(os.path.join(plotsavepath,'loss_seed3.jpg'))
# plt.show()

In [12]:
sam = adata.obsm['feat']

In [13]:
com = features_recon[3].detach().cpu().numpy() 

In [19]:
# plt.style.use('dark_background')
# for i in range(20):
#     fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(8,4))
#     ax1.axis('off')
#     cmap = sam[:,i]
#     cmap[cmap > np.percentile(cmap,99)] = np.percentile(cmap,99)
#     ax1.scatter(adata.obsm['spatial'][:,0],adata.obsm['spatial'][:,1],s=1,c=cmap)
#     ax1.set_title('Measured ', fontsize = 12)
#     ax1.set_ylabel(i)
#     ax1.invert_yaxis()
#     # ax1.invert_xaxis()
#     ax2.axis('off')
#     cmap = com[:,i]
#     cmap[cmap > np.percentile(cmap,99)] = np.percentile(cmap,99)
#     ax2.scatter(adata.obsm['spatial'][:,0],adata.obsm['spatial'][:,1],s=1,c=cmap)
#     ax2.set_title('Predicted ', fontsize = 12)
#     ax2.invert_yaxis()
#     # ax2.invert_xaxis()

In [14]:
sam = anndata.AnnData(sam,var = adata[:,adata.var['highly_variable']].var)
com = anndata.AnnData(com)

In [21]:
# import pandas as pd
# import scipy.stats as st

# def cal_Percor(original,res):
#     Pearson_CoPearson_Cor = pd.Series(index=original.var_names)
    
#     for i in range(res.X.shape[1]):
#         Pearson_CoPearson_Cor[i]=st.pearsonr(original.X.T[i],res.X.T[i])[0]
#     Pearson_Cor_mean = np.mean(Pearson_CoPearson_Cor)
#     return Pearson_CoPearson_Cor,Pearson_Cor_mean
# our_Percor,our_Percor_mean = cal_Percor(sam,com)
# print(our_Percor_mean)
# #0.3171313503053768
# #0.34746957606266593
# #0.408

In [15]:
def cal_Percor(original,res):
    Pearson_CoPearson_Cor = pd.Series(index=original.var_names)
    for i in range(res.X.shape[1]):
        Pearson_CoPearson_Cor[i]=st.pearsonr(original.X[i],res.X[i])[0]
    Pearson_Cor_mean = np.mean(Pearson_CoPearson_Cor)
    return Pearson_CoPearson_Cor,Pearson_Cor_mean
our_Percor,our_Percor_mean = cal_Percor(sam,com)
print(our_Percor_mean)
#0.664

/tmp/ipykernel_923019/448874403.py:4: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  Pearson_CoPearson_Cor[i]=st.pearsonr(original.X[i],res.X[i])[0]


0.570350098308893


In [23]:
# def cal_Specor(original,res):
#     Spearman_CoPearson_Cor = pd.Series(index=original.var_names)
#     for i in range(res.X.shape[1]):
#         Spearman_CoPearson_Cor[i]=st.spearmanr(original.X[i],res.X[i])[0]
#     Spearman_Cor_mean = np.mean(Spearman_CoPearson_Cor)
#     return Spearman_CoPearson_Cor,Spearman_Cor_mean
# our_Specor,our_Specor_mean = cal_Specor(sam,com)
# print(our_Specor_mean)
# #0.598

In [15]:
# # 进行主成分分析（PCA）
# com.uns['spatial']=adata.uns['spatial']
# com.obsm['spatial'] = adata.obsm['spatial']
# sc.pp.pca(com, svd_solver="arpack")
# # 绘制主成分方差解释曲线
# sc.pp.neighbors(com, n_neighbors=10,n_pcs=30)
# sc.tl.umap(com)
# sc.tl.leiden(com, key_added="leiden_res", resolution=0.25)
# sc.pl.umap(
#     com,
#     # color=["leiden_res0_15", "leiden_res0_5", "leiden_res0_75", "leiden_res1"],
#     color=["leiden_res"],
#     legend_loc="on data",
# )
# # sc.pl.spatial(our,img_key='hires',color=["leiden_res0_15", "leiden_res0_5", "leiden_res0_75", "leiden_res1"])
# sc.pl.spatial(com,img_key='hires',color=["leiden_res"])

In [27]:
# from sklearn import metrics
# score1 = metrics.silhouette_score(com.obsm['X_pca'][:,0:25],labels=com.obs['leiden_res'])
# print(score1)
# score2 = metrics.silhouette_score(com.obsm['spatial'],labels=com.obs['leiden_res'])
# print(score2)

0.19248872
0.15438679715376752


In [16]:
result = anndata.AnnData(com)
result.write(os.path.join("./outputs/human_breast_cancer", f"ourfuc_drop_new050.h5ad"))